### Two Strategies:
1. compress hsi bands to 3
    - PCA
2. process 3 bands a time, and merge the information in final layer for segmentation.

In [1]:
from osgeo import gdal
gdal.UseExceptions()
import rasterio
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from imageio import imwrite
import numpy as np


### Read HSI Value 

In [15]:
# Open the HSI TIFF file
dataset = gdal.Open('stacked_hsi.tif', gdal.GA_ReadOnly)

# Specify the pixel coordinates you are interested in (x, y)
x, y = 10, 10  # for example

# Loop through each band and read the pixel value
pixel_spectral_values = []
for b in range(dataset.RasterCount):
    band = dataset.GetRasterBand(b + 1)  # Band count starts at 1
    pixel_value = band.ReadAsArray(x, y, 1, 1)
    pixel_spectral_values.append(pixel_value[0][0])

print(f"Spectral values at pixel ({x}, {y}):", pixel_spectral_values)

Spectral values at pixel (10, 10): [15360, 25696, 12608, 35728, 38368]


### Load HSI

In [2]:
# Load the hyperspectral image
with rasterio.open('stacked_hsi.tif') as src:
    img = src.read()

# Reshape the image for PCA; (bands, rows, cols) 
print('original_shape: (bands, rows, cols) =', img.shape)
print('original value at pixel (10, 10):', img[0][10][10], img[1][10][10], img[2][10][10], img[3][10][10], img[4][10][10])

original_shape: (bands, rows, cols) = (5, 960, 1280)
original value at pixel (10, 10): 15360 25696 12608 35728 38368


### Standardize HSI values

In [3]:
bands, rows, cols = img.shape

# Transpose img to bring bands to the last dimension: new order (rows, columns, bands)
img_transposed = np.transpose(img, (1, 2, 0))

# Reshape the image to have one pixel per row, and bands as columns
pixels = img.reshape(-1, bands)

# Calculate mean and standard deviation for each band
mean = np.mean(pixels, axis=0)
std = np.std(pixels, axis=0)

# Standardize the pixels
standardized_pixels = (pixels - mean) / std

# Reshape back to original dimensions if necessary
standardized_hsi_img = standardized_pixels.reshape(bands, rows, cols)

# Now 'standardized_hsi_image' is the standardized hyperspectral image
print('value after standardization at pixel (10, 10):', standardized_hsi_img[0][10][10], standardized_hsi_img[1][10][10], standardized_hsi_img[2][10][10], standardized_hsi_img[3][10][10], standardized_hsi_img[4][10][10])


value after standardization at pixel (10, 10): -0.9572107742415521 -0.1846613443969822 -1.1629050496800444 0.5651660433933356 0.7624890401802614


### reshape the original HSI 

In [4]:
original_shape = img.shape
# Reshape to (n_samples, n_features)
hsi_reshape = img.reshape(img.shape[0], -1).T
# 960 * 1280 = 1228800
print('after reshape:', hsi_reshape.shape)

after reshape: (1228800, 5)


### Using PCA
- Principal Component Analysis: statistical technique used for dimensionality reduction while preserving as much of the data’s variability as possible.

### Normalization vs Standardization
Outlier Sensitivity: 
Normalization can be sensitive to outliers since the minimum and maximum values are used for scaling. Standardization, on the other hand, is less sensitive to outliers because it uses the mean and standard deviation, which are less influenced by extreme values.

In [80]:
# Apply PCA
pca = PCA(n_components=3)
pca_reduced = pca.fit_transform(hsi_reshape)

# Reshape back to (3, rows, cols)
pca_reshaped = pca_reduced.T.reshape(3, original_shape[1], original_shape[2])

# Normalize and convert data to uint8
normalized_pca = ((pca_reshaped - pca_reshaped.min()) / (pca_reshaped.max() - pca_reshaped.min()) * 255).astype('uint8')

### Save as RGB

In [81]:
# Save the first three bands as an RGB image
imwrite('pca_reduced.png', normalized_pca.transpose(1, 2, 0))  # Transpose to get shape (rows, cols, bands)

### Independent Component Analysis(ICA)

In [85]:
from sklearn.decomposition import FastICA

# Apply ICA
ica = FastICA(n_components=3)
ICA_reduced = ica.fit_transform(hsi_reshape)

# Reshape and process like PCA
ICA_reshaped = ICA_reduced.T.reshape(3, original_shape[1], original_shape[2])

# Normalize and convert data to uint8
normalized_ica = ((ICA_reshaped - ICA_reshaped.min()) / (ICA_reshaped.max() - ICA_reshaped.min()) * 255).astype('uint8')

### save as RGB

In [86]:
# Save the first three bands as an RGB image
imwrite('ica_reduced.png', normalized_ica.transpose(1, 2, 0))  # Transpose to get shape (rows, cols, bands)

### Discrete Cosine Transform (DCT) - Fourier transform 

In [6]:
from scipy.fftpack import dct, idct

def apply_dct(image):
    return dct(dct(image.T, norm='ortho').T, norm='ortho')

def apply_idct(image):
    return idct(idct(image.T, norm='ortho').T, norm='ortho')

# Apply DCT to each band
dct_images = np.array([apply_dct(band) for band in img])

# Optionally, apply inverse DCT to see reconstruction
reconstructed_images = np.array([apply_idct(band) for band in dct_images])

In [7]:
# Set threshold for compression
threshold = np.quantile(dct_images, 0.95)

# Zero out small DCT coefficients
compressed_dct_images = np.where(np.abs(dct_images) < threshold, 0, dct_images)

# Apply inverse DCT to compressed image
compressed_images = np.array([apply_idct(band) for band in compressed_dct_images])

In [8]:
# Define metadata and transformation (assuming same as input)
with rasterio.open('stacked_hsi.tif') as src:
    meta = src.meta.copy()

# Update meta to reflect any changes in dtype or number of bands
meta.update(dtype='float32')

# Save compressed image
with rasterio.open('dct_compressed.tif', 'w', **meta) as dst:
    dst.write(compressed_images.astype('float32'))

/opt/anaconda3/envs/opencv/lib/python3.12/site-packages/rasterio/__init__.py:366: NotGeoreferencedWarning: The given matrix is equal to Affine.identity or its flipped counterpart. GDAL may ignore this matrix and save no geotransform without raising an error. This behavior is somewhat driver-specific.
  dataset = writer(


In [23]:
# Example: Select bands at index 0, 1, 2
rgb_image = compressed_images[[0, 2, 4], :, :]

# Normalize data to 0-255
def normalize_data(img):
    img_min = np.min(img)
    img_max = np.max(img)
    return ((img - img_min) / (img_max - img_min) * 255).astype(np.uint8)

# Apply normalization to each band
rgb_normalized = np.array([normalize_data(band) for band in rgb_image])

# Transpose to (height, width, channels) format required by most image libraries
rgb_normalized = rgb_normalized.transpose(1, 2, 0)

In [24]:
# Save the RGB image as a PNG file
imwrite('dct_compressed_024.png', rgb_normalized)

### Weighted Sum of bands

In [ ]:
# Define weights - simple linear example
num_bands = img.shape[0]
# weights = np.linspace(1, 2, num_bands)  # Linearly increasing weights from 1 to 2
# TODO: make them parameters - learnable. 
weights = np.array([0.5, 0.1, 0.2, 0.1, 0.1])  # Custom weights

# Apply weights and sum along the band dimension
weighted_sum = np.tensordot(weights, standardized_hsi_img, axes=[0, 0])

In [32]:
print('weights:', weights)
print('weighted_sum:', weighted_sum)

weights: [0.5 0.1 0.2 0.1 0.1]
weighted_sum: [[-0.6505828  -0.63230115 -0.6556062  ... -0.87544827 -0.94637951
  -1.06246386]
 [-0.62200086 -0.65514113 -0.65273557 ... -0.98046532 -1.00845857
  -1.09441119]
 [-0.68394832 -0.65011873 -0.6670887  ... -0.97173384 -1.04171095
  -0.99509928]
 ...
 [-0.72783774 -0.76336198 -0.75942716 ... -0.89051906 -0.85619142
  -0.86479726]
 [-0.69483098 -0.69567911 -0.73658176 ... -0.85021069 -0.80858551
  -0.83943084]
 [-0.6923196  -0.71325752 -0.73765825 ... -0.84423022 -0.84590471
  -0.79384308]]


In [34]:
# Normalize the weighted sum to fit into a typical image range
weighted_sum_normalized = 255 * (weighted_sum - np.min(weighted_sum)) / (np.max(weighted_sum) - np.min(weighted_sum))
weighted_sum_normalized = weighted_sum_normalized.astype(np.uint8)

# Save resulting image
imwrite('customized_weighted_sum.png', weighted_sum_normalized)

Yes, you can process three bands at a time from a multi-band hyperspectral image (HSI) and combine the results at the final layer. This approach is a common strategy in processing hyperspectral data, especially when trying to manage the high dimensionality and correlation between spectral bands. Here’s a general outline of how you might implement this:
1.	Band Selection:
	•	Select subsets of bands, such as three bands at a time, that you believe contain significant information for your analysis. The selection of bands can be sequential, random, or based on some spectral characteristics like bands covering specific wavelengths of interest.
2.	Feature Extraction:
	•	Process each triplet of bands separately through a neural network or another machine learning model to extract features. This can involve convolutional layers if you are using deep learning, which are adept at capturing spatial and spectral features.
3.	Combining Features:
	•	After processing the individual triplets, the next step is to combine the extracted features. This can be done in several ways:
	•	Concatenation: The features from each triplet are concatenated into a single feature vector.
	•	Feature Fusion: Use more sophisticated methods like averaging, maximum, or learned fusion methods to combine features.
4.	Final Layers:
	•	Pass the combined features through additional layers (if needed) to make final predictions or analyses. This could be classification layers, regression layers, or other custom layers depending on your application.
5.	Training and Inference:
	•	During training, ensure that the network learns to effectively integrate the features from different band triplets. It might be beneficial to include regularization techniques to prevent overfitting given the high-dimensional nature of HSI data.
	•	Inference would follow the trained model’s capability to generalize from the learned features of band triplets to unseen data.